We want to find a representative file that was updated for Quarkus which we can use for an example
Consider:  
* https://github.com/mathianasj/eap-coolstore-monolith/blob/quarkus-migration/src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java
* https://github.com/mathianasj/eap-coolstore-monolith/blob/quarkus-migration/src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java

The larger idea we are working towards is:
1) Find a file which has been changed and get the original + diff
2) Find a similar file we need to update, and get the original of that file
3) Send a request to LLM and see how well it does with a patch
4) Compare it to the diff we have

Documentation for library: https://gitpython.readthedocs.io/en/stable/quickstart.html#trees-blobs

We want to learn how to generate a patch from a diff.
Reading API docs at: https://gitpython.readthedocs.io/en/stable/reference.html#module-git.diff

Assume 'create_patch' will help.

`create_patch – If True, the returned Diff contains a detailed patch that if applied makes the self to other. Patches are somewhat costly as blobs have to be read and diffed.`

Looking at this git history: 
* https://github.com/mathianasj/eap-coolstore-monolith/commits/quarkus-migration/src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java

In [ ]:
# Playing around to learn the library
from git import Repo

example_git_path = "../data/coolstuff-quarkus"
example_file = "src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java"

# We want to find the commits associated to a specific file in the Git Repo
repo = Repo.init(example_git_path)
commits_for_file_generator = repo.iter_commits(all=True, max_count=10, paths=example_file)
commits_for_file = list(commits_for_file_generator)
print(f"{example_file} has {len(commits_for_file)} commits associated")
print(f"{commits_for_file}")

In our example for the given file:
* latest commit is in commits_for_file[0]
* original commit is in commits_for_file[1]

In [ ]:
print(f"The last commit on the file `{example_file}` is `{commits_for_file[0]}`")
print(commits_for_file[0].stats.files[example_file])

In [ ]:
print(f"The original commit on the file `{example_file}` is `{commits_for_file[1]}`")
print(commits_for_file[1].stats.files[example_file])

Lets look at the actual contents of the example file in git for it's latest commit

In [ ]:
original_tree = repo.tree(commits_for_file[1])
updated_tree = repo.tree(commits_for_file[0])

blob = updated_tree[example_file]
print(f"Contents of {example_file} at HEAD")
print(f"\n---HEAD---")
print(blob.data_stream.read().decode())
print(f"\n-----\n")

Exploring some error behaviors


In [13]:
from git import BadName 

try:
    repo.commit("bad_id")
except BadName as e:
    print(f"We caught Bad Name: {e}") 

Repo is <git.repo.base.Repo '/Users/jmatthews/git/jwmatthews/kyma_poc/nbs/bad_path/.git'>
We caught Bad Name: Ref 'bad_id' did not resolve to an object


We will create a class next we can reuse to get the diff for a given file

In [15]:
#| default_exp scm
#| export

from git import BadName, Repo

class GitDiff:
    def __init__(self, repo_path):
        self.repo_path = repo_path
        self.repo = Repo(self.repo_path) 

    def get_patch(self, start_commit_id, end_commit_id="HEAD"):
        # If either commit_id is not valid, this will raise a BadName exception
        start_commit = self.repo.commit(start_commit_id)
        end_commit = self.repo.commit(end_commit_id)
        return start_commit.diff(end_commit_id, create_patch=True) 
        
    def get_patch_for_file(self, start_commit_id, end_commit_id, file_path):
        diff_indexes = self.get_patch(start_commit_id, end_commit_id)
        # We need to search through the indexes to find the diff for our file_path
        patch = None
        for diff in diff_indexes:
            print(f"'{file_path}' '{diff.a_path}' '{diff.b_path}' ")
            if diff.a_path == file_path or diff.b_path == file_path:
                print("Found match")
                patch = diff.diff
                break
        return patch


In [18]:
gd = GitDiff(example_git_path)
#Remember that the original state of the file is in the second commit
#The current state of the file is in the first commit
patch = gd.get_patch_for_file(commits_for_file[1].hexsha, commits_for_file[0].hexsha, example_file)

if patch:
    sPatch = patch.decode('utf-8')
    print(f'Patch for {example_file} between {commits_for_file[1].hexsha} and {commits_for_file[0].hexsha}:\n{sPatch}')
    changed_lines = sPatch.split('\n')[2:]
    additions = sum(1 for line in changed_lines if line.startswith('+'))
    deletions = sum(1 for line in changed_lines if line.startswith('-'))
    print(f'Changes: {additions} additions, {deletions} deletions')
    # For the example we chose, there are only 2 commits
    # so we can assume that the diff from latest commit and 
    # first commit will match from our diff and what is in the 
    # assertion below
    assert additions == commits_for_file[0].stats.files[example_file]['insertions']
    assert deletions == commits_for_file[0].stats.files[example_file]['deletions']
else:
    print(f'No changes in {example_file} between {commits_for_file[1].hexsha} and {commits_for_file[0].hexsha}')


'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'None' '.github/workflows/main_clustertesting(dev).yml' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'None' '.vscode/settings.json' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'README.md' 'README.md' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'pom.xml' 'pom.xml' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'None' 'realm-export.json' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'src/main/java/com/redhat/coolstore/model/CatalogItemEntity.java' 'src/main/java/com/redhat/coolstore/model/CatalogItemEntity.java' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'src/main/java/com/redhat/coolstore/model/InventoryEntity.java' 'src/main/java/com/redhat/coolstore/model/InventoryEntity.java' 
'src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java' 'src/main/java/com/redhat/coolstore/model/Order